# Model answers to Exam 1

*Last updated by Christian Cahig on 2025-11-11*

In [1]:
import math as mt

import scipy.optimize as spo

In [ ]:
DEG2RAD = mt.pi / 180.
RAD2DEG = 180. / mt.pi

P1s = {
    "M34W12" : 0.450,   # in megawatts
    "M34W34" : 0.375,   # in megawatts
    "H34W56" : 0.425,   # in megawatts
    "H34W78" : 0.301,   # in megawatts
}
PHI1s = {
    "M34W12" : 0.87,
    "M34W34" : 0.95,
    "H34W56" : 0.75,
    "H34W78" : 0.73,
}
Q2s = {
    "M34W12" : 0.300,   # in megavars
    "M34W34" : 0.325,   # in megavars
    "H34W56" : 0.275,   # in megavars
    "H34W78" : 0.334,   # in megavars
}
PHI2s = {
    "M34W12" : 0.90,
    "M34W34" : 0.80,
    "H34W56" : 0.75,
    "H34W78" : 0.84,
}

R1s = {
    "M34W12" : 0.970,   # in ohms
    "M34W34" : 0.950,   # in ohms
    "H34W56" : 1.010,   # in ohms
    "H34W78" : 0.990,   # in ohms
}
X1s = {
    "M34W12" : 2.450,   # in ohms
    "M34W34" : 2.540,   # in ohms
    "H34W56" : 2.750,   # in ohms
    "H34W78" : 2.510,   # in ohms
}
R2s = {
    "M34W12" : 0.990,   # in ohms
    "M34W34" : 1.010,   # in ohms
    "H34W56" : 0.950,   # in ohms
    "H34W78" : 0.970,   # in ohms
}
X2s = {
    "M34W12" : 2.510,   # in ohms
    "M34W34" : 2.750,   # in ohms
    "H34W56" : 2.540,   # in ohms
    "H34W78" : 2.450,   # in ohms
}

Us = {
    "M34W12" : 49.00,   # in kilovolts
    "M34W34" : 13.80,   # in kilovolts
    "H34W56" : 13.80,   # in kilovolts
    "H34W78" : 49.00,   # in kilovolts
}
U_factors = {
    "M34W12" : 0.90,
    "M34W34" : 1.10,
    "H34W56" : 0.90,
    "H34W78" : 1.10,
}

# Helper functions
def get_S_from_P_Phi(P, Phi, is_lagging = True):
    return (
        P / Phi,
        mt.acos(Phi) if is_lagging else -mt.acos(Phi)
    )

def get_S_from_Q_Phi(Q, Phi, is_lagging = True):
    return (
        Q / mt.sin(mt.acos(Phi)),
        mt.acos(Phi) if is_lagging else -mt.acos(Phi)
    )

def get_Z_from_R_X(R, X):
    return (
        mt.sqrt(R**2 + X**2),
        mt.atan2(X, R)
    )

In [ ]:
SECTION = "H34W78"

s1, theta1 = get_S_from_P_Phi(P1s[SECTION], PHI1s[SECTION], is_lagging=True)
s2, theta2 = get_S_from_Q_Phi(Q2s[SECTION], PHI2s[SECTION], is_lagging=True)
z1, alpha1 = get_Z_from_R_X(R1s[SECTION], X1s[SECTION])
z2, alpha2 = get_Z_from_R_X(R2s[SECTION], X2s[SECTION])
u = U_factors[SECTION] * Us[SECTION]

In [4]:
def f(v, v_ss = Us[SECTION]):
    _v2 = v**2
    _sz = s1 * z1
    return ((
        (2 * _sz * mt.cos(alpha1 - theta1)) - (v_ss**2)
    ) * _v2) + (_v2**2) + (_sz)**2

In [5]:
def g(v, v_ss = Us[SECTION]):
    _v2 = v**2
    _sz = s2 * z2
    return ((
        (2 * _sz * mt.cos(alpha2 - theta2)) - (v_ss**2)
    ) * _v2) + (_v2**2) + (_sz)**2

In [6]:
def df(v, v_ss = Us[SECTION]):
    return (4 * (v**3)) + (2 * (
        (2 * s1 * z1 * mt.cos(alpha1 - theta1)) - (v_ss**2)
    ))

In [7]:
def dg(v, v_ss = Us[SECTION]):
    return (4 * (v**3)) + (2 * (
        (2 * s2 * z2 * mt.cos(alpha2 - theta2)) - (v_ss**2)
    ))

In [8]:
def calc_angle_diff(v1, v2):
    _szv = (s1 * z1) / v1
    _alth = alpha1 - theta1
    delta0 = mt.atan2(
        _szv * mt.sin(_alth),
        v1 + (_szv * mt.cos(_alth))
    )
    
    _szv = (s2 * z2) / v2
    _alth = alpha2 - theta2
    delta0_bar = mt.atan2(
        _szv * mt.sin(_alth),
        v2 + (_szv * mt.cos(_alth))
    )

    return delta0_bar - delta0

### Using the bisection method

In [9]:
_V1L, _V1U = 0.3 * u, u
_V2L, _V2U = 0.3 * u, u

v1_bs, v1_bs_info = spo.bisect(
    f, _V1L, _V1U, args=(u,),
    xtol=1e-13, maxiter=100,
    full_output=True, disp=False
)
v2_bs, v2_bs_info = spo.bisect(
    g, _V2L, _V2U, args=(u,),
    xtol=1e-13, maxiter=100,
    full_output=True, disp=False
)

In [10]:
print("-----")
print(v1_bs_info)
print("-----")
print(v2_bs_info)

-----
      converged: True
           flag: converged
 function_calls: 50
     iterations: 48
           root: 53.88134123210408
         method: bisect
-----
      converged: True
           flag: converged
 function_calls: 50
     iterations: 48
           root: 53.875498702138835
         method: bisect


In [11]:
print(f"Solving for Bonnie's operating voltage using the bisection method...")
print(f"\tinterval: {_V1L} - {_V1U} kilovolts")
print(f"\troot: {v1_bs}")
print(f"\tresidual: {f(v1_bs, v_ss=u)}")

Solving for Bonnie's operating voltage using the bisection method...
	interval: 16.17 - 53.900000000000006 kilovolts
	root: 53.88134123210408
	residual: 1.4696877714825973e-08


In [12]:
print(f"Solving for Clyde's operating voltage using the bisection method...")
print(f"\tinterval: {_V2L} - {_V2U} kilovolts")
print(f"\troot: {v2_bs}")
print(f"\tresidual: {g(v2_bs, v_ss=u)}")

Solving for Clyde's operating voltage using the bisection method...
	interval: 16.17 - 53.900000000000006 kilovolts
	root: 53.875498702138835
	residual: -1.0621592494430843e-09


In [13]:
print(f"Phase-angle difference: {calc_angle_diff(v1_bs, v2_bs) * DEG2RAD} degrees")

Phase-angle difference: 2.8031630047716273e-06 degrees


### Using the Newton-Raphson method

In [14]:
_V1NR, _V2NR = v1_bs, v2_bs

v1_nr, v1_nr_info = spo.newton(
    f, _V1NR, fprime=df, args=(u,),
    full_output=True, disp=False
)

v2_nr, v2_nr_info = spo.newton(
    g, _V2NR, fprime=dg, args=(u,),
    full_output=True, disp=False
)

In [15]:
print("-----")
print(v1_nr_info)
print("-----")
print(v2_nr_info)

-----
      converged: True
           flag: converged
 function_calls: 2
     iterations: 1
           root: 53.881341232104056
         method: newton
-----
      converged: True
           flag: converged
 function_calls: 2
     iterations: 1
           root: 53.875498702138835
         method: newton


In [16]:
print(f"Solving for Bonnie's operating voltage using the Newton-Raphson method...")
print(f"\tstart: {_V1NR} kilovolts")
print(f"\troot: {v1_nr}")
print(f"\tresidual: {f(v1_nr, v_ss=u)}")

Solving for Bonnie's operating voltage using the Newton-Raphson method...
	start: 53.88134123210408 kilovolts
	root: 53.881341232104056
	residual: 7.246297117902145e-09


In [17]:
print(f"Solving for Clyde's operating voltage using the Newton-Raphson method...")
print(f"\tstart: {_V2NR} kilovolts")
print(f"\troot: {v2_nr}")
print(f"\tresidual: {g(v2_nr, v_ss=u)}")

Solving for Clyde's operating voltage using the Newton-Raphson method...
	start: 53.875498702138835 kilovolts
	root: 53.875498702138835
	residual: -1.0621592494430843e-09


In [18]:
print(f"Phase-angle difference: {calc_angle_diff(v1_nr, v2_nr) * DEG2RAD} degrees")

Phase-angle difference: 2.8031630047716247e-06 degrees


### Using the secant method

In [19]:
_V1SC, _V2SC = v1_bs, v2_bs

v1_sc, v1_sc_info = spo.newton(
    f, _V1SC, args=(u,),
    tol=1e-13, maxiter=100,
    full_output=True, disp=False
)

v2_sc, v2_sc_info = spo.newton(
    g, _V2SC, args=(u,),
    tol=1e-13, maxiter=100,
    full_output=True, disp=False
)

In [20]:
print("-----")
print(v1_sc_info)
print("-----")
print(v2_sc_info)

-----
      converged: True
           flag: converged
 function_calls: 3
     iterations: 2
           root: 53.881341232104035
         method: secant
-----
      converged: True
           flag: converged
 function_calls: 3
     iterations: 2
           root: 53.87549870213884
         method: secant


In [21]:
print(f"Solving for Bonnie's operating voltage using the secant method...")
print(f"\tstart: {_V1SC} kilovolts")
print(f"\troot: {v1_sc}")
print(f"\tresidual: {f(v1_sc, v_ss=u)}")

Solving for Bonnie's operating voltage using the secant method...
	start: 53.88134123210408 kilovolts
	root: 53.881341232104035
	residual: 1.658361670209274e-09


In [22]:
print(f"Solving for Clyde's operating voltage using the secant method...")
print(f"\tstart: {_V2SC} kilovolts")
print(f"\troot: {v2_sc}")
print(f"\tresidual: {g(v2_sc, v_ss=u)}")

Solving for Clyde's operating voltage using the secant method...
	start: 53.875498702138835 kilovolts
	root: 53.87549870213884
	residual: 8.004858997878728e-10


In [23]:
print(f"Phase-angle difference: {calc_angle_diff(v1_sc, v2_sc) * DEG2RAD} degrees")

Phase-angle difference: 2.80316300477162e-06 degrees


In [24]:
print(v1_bs, v1_nr, v1_sc)
print(f(v1_bs, v_ss=u), f(v1_nr, v_ss=u), f(v1_sc, v_ss=u))

53.88134123210408 53.881341232104056 53.881341232104035
1.4696877714825973e-08 7.246297117902145e-09 1.658361670209274e-09


In [25]:
print(v2_bs, v2_nr, v2_sc)
print(g(v2_bs, v_ss=u), g(v2_nr, v_ss=u), g(v2_sc, v_ss=u))

53.875498702138835 53.875498702138835 53.87549870213884
-1.0621592494430843e-09 -1.0621592494430843e-09 8.004858997878728e-10


In [26]:
print(df(v1_bs, v_ss=u), df(v1_nr, v_ss=u), df(v1_sc, v_ss=u))

619906.6107696263 619906.6107696256 619906.6107696248
